What we need is a small parcel to work on, with a nice contourplot with critical points and minima found after initiating local method and then a 3d plot. 

In [5]:
using Pkg
Pkg.activate("../../.")
using Globtim
using DynamicPolynomials, DataFrames
using ProgressLogging
using Optim

  Activating project at `~/globtim`


In [6]:
# Constants and Parameters
const n, a, b = 2, 12, 10
const scale_factor = a / b   # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = 0.5, 1 / 10  # Sampling parameters
const tol_l2 = 3e-4            # Define the tolerance for the L2-norm
f = Deuflhard # Objective function

Deuflhard (generic function with 1 method)

In [7]:
f([0,0])

4.0

One may assume that when we have access to exact evaluations, we would want to have a small $L^2$-norm tolerance `tol_l2 = 5e-4` and high probability of computing an accurate discrete $L^2$-norm `alpha= 1/10`.

We need to also return the number of samples used to generate the sample set. It is annoying that the error goes up while the degree has increased.

In [7]:
d = 8 # Initial Degree 
SMPL = 40 # Number of samples
TR = test_input(f, 
                dim = n,
                center=[0.0, 0.0],
                GN=SMPL 
                )
pol_cheb = Constructor(TR, d, basis=:chebyshev)
pol_lege = Constructor(TR, d, basis=:legendre);

@polyvar(x[1:n]); # Define polynomial ring 

current L2-norm: 0.5289850410278208
current L2-norm: 0.49248824331534263


In [8]:
df_cheb = solve_and_parse(pol_cheb, x, f, TR)
sort!(df_cheb, :z, rev=true)
df_lege = solve_and_parse(pol_lege, x, f, TR, basis=:legendre)
sort!(df_lege, :z, rev=true)

df_cheb, df_min_cheb = analyze_critical_points(f, df_cheb, TR, tol_dist=1.0)
df_lege, df_min_lege = analyze_critical_points(f, df_lege, TR, tol_dist=1.0)

Dimension m of the vector space: 45

=== Starting MSolve Parser (dimension: 2) ===
Processed 13 points (0.0s)
Dimension m of the vector space: 45

=== Starting MSolve Parser (dimension: 2) ===
Processed 13 points (0.0s)
Processing point 1 of 9
Optimization has converged within bounds: ✓
Processing point 2 of 9
Optimization status: ✗ (outside bounds)
Processing point 3 of 9
Optimization status: ✗ (outside bounds)
Processing point 4 of 9
Optimization status: ✗ (outside bounds)
Processing point 5 of 9
Optimization status: ✗ (outside bounds)
Processing point 6 of 9
Optimization status: ✗ (outside bounds)
Processing point 7 of 9
Optimization status: ✗ (outside bounds)
Processing point 8 of 9
Optimization has converged within bounds: ✓
Processing point 9 of 9
Optimization has converged within bounds: ✓
Processing point 1 of 9
Optimization has converged within bounds: ✓
Processing point 2 of 9
Optimization has converged within bounds: ✓
Processing point 3 of 9
Optimization status: ✗ (outside 

(9×8 DataFrame
 Row │ x1            x2            z           y1            y2            clo ⋯
     │ Float64       Float64       Float64     Float64       Float64       Boo ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ -3.90326e-17   6.88014e-16  4.0         -3.90326e-17   6.88014e-16   tr ⋯
   2 │ -0.410543     -0.410543     2.5947      -0.459896     -0.459896      tr
   3 │  0.410543      0.410543     2.5947      -0.256625      1.01625       tr
   4 │ -0.453474      0.93908      0.259065    -0.256625      1.01625       tr
   5 │  0.453474     -0.93908      0.259065     0.256625     -1.01625       tr ⋯
   6 │ -0.93908       0.453474     0.259065    -1.01625       0.256625      tr
   7 │  0.93908      -0.453474     0.259065     1.01625      -0.256625      tr
   8 │  0.750171     -0.750171     0.00669131   0.741152     -0.741152      tr
   9 │ -0.750171      0.750171     0.00669131  -0.741152      0.741152      tr ⋯
                         

Solve the system of partial derivatives using `Msolve`. 

In [15]:
df_cheb = msolve_parser("outputs.ms", f, scale_factor, n)

MethodError: MethodError: no method matching msolve_parser(::String, ::typeof(Deuflhard), ::Float64, ::Int64)
The function `msolve_parser` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  msolve_parser(::String, ::Function, !Matched::test_input)
   @ Globtim ~/globtim/src/ParsingOutputs.jl:49


In [7]:
using GLMakie

# Extract coordinates and function values
# Change the coordinates to uniform grid
coords = pol_cheb.scale_factor * pol_cheb.grid
z_coords = pol_cheb.z

if size(coords)[2] == 2  # Plot if the dimensions are 2
    fig = Figure(size=(800, 600))
    ax = Axis3(fig[1, 1], title="Deuflhard Function",
        xlabel="X-axis", ylabel="Y-axis", zlabel="Z-axis")

    # Scale the domain
    scale_factor = pol_cheb.scale_factor  # Using the scale factor from your polynomial

    # Define threshold for switching between scatter and surface
    point_threshold = 1000  # Adjust this value based on your needs
    sample_fraction = 0.2   # Fraction of points to use if exceeding threshold

    if length(z_coords) > point_threshold
        # Create a regular grid for surface plotting
        x_unique = sort(unique(coords[:, 1]))
        y_unique = sort(unique(coords[:, 2]))

        # Determine grid dimensions
        nx = length(x_unique)
        ny = length(y_unique)

        # Reshape data into a grid
        z_grid = reshape(z_coords, nx, ny)

        # Create surface plot with scaled domain
        surface!(ax, scale_factor .* x_unique, scale_factor .* y_unique, z_grid,
            colormap=:viridis,
            transparency=true,
            alpha=0.8)

        # Sample a fraction of points for scatter overlay
        n_points = length(z_coords)
        sample_indices = rand(1:n_points, Int(floor(n_points * sample_fraction)))

        # Plot sampled scatter points with scaled domain
        scatter!(ax, scale_factor .* coords[sample_indices, 1],
            scale_factor .* coords[sample_indices, 2],
            z_coords[sample_indices],
            markersize=2,
            color=:black,
            label="Sampled Data Points")
    else
        # Original scatter plot for smaller datasets with scaled domain
        scatter!(ax, scale_factor .* coords[:, 1],
            scale_factor .* coords[:, 2],
            z_coords,
            markersize=2,
            color=:black,
            label="Sampled Data Core")

        scatter!(ax, scale_factor .* coords[:, 1],
            scale_factor .* coords[:, 2],
            z_coords,
            markersize=4,
            color=z_coords,
            colormap=:viridis,
            label="Sampled Data Halo")
    end

    # Plot the critical points with scaled domain
    scatter!(ax, scale_factor .* df_cheb.x1,
        scale_factor .* df_cheb.x2,
        df_cheb.z,
        markersize=10,
        color=:orange,
        label="Chebyshev approximant critical points")

    # scatter!(ax, scale_factor .* df_lege.x1,
    #     scale_factor .* df_lege.x2,
    #     df_lege.z,
    #     markersize=10,
    #     color=:yellow,
    #     label="Legendre approximant critical points")

    display(fig)
end

GLMakie.Screen(...)